In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

#Statsmodels is a great library we can use to run regressions.
import statsmodels.api as sm
# Seaborn extends the capabilities of Matplotlib
import seaborn as sns
# Used for calculating regressions
from statsmodels.tsa.ar_model import AutoReg, ar_select_order

## Dates & Other Constants

In [2]:
# Define path to files
# For MacOS
# PATH = "/Users/derekbanas/Documents/Tutorials/Python for Finance/Stocks/"
# For Windows
PATH = "D:/Python for Finance/Stocks/"

# Start date defaults
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = "2017-01-03"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2021
E_MONTH = 8
E_DAY = 19
E_DATE_STR = "2021-08-19"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

risk_free_rate = 0.0125 # Approximate 10 year bond rate

## Function that Saves Dataframe to CSV

In [3]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

## Function that Returns a Dataframe from a CSV

In [4]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col='Date', 
                         parse_dates=True)
    except FileNotFoundError:
        pass
        # print("File Doesn't Exist")
    else:
        return df

## Add Daily Return to Dataframe

In [5]:
# Shift provides the value from the previous day
# NaN is displayed because there was no previous day price for the 1st calculation
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    # Save data to a CSV file
    save_dataframe_to_csv(df, ticker)
    return df  

## Merge Multiple Stocks in One Dataframe by Column Name

In [43]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        
        # NEW Check if your dataframe has duplicate indexes
        if not df.index.is_unique:
            # Delete duplicates 
            df = df.loc[~df.index.duplicated(), :]
        
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

In [44]:
port_list = ["AMD", "CPRT"]
mult_df = merge_df_by_column_name('daily_return',  '2018-01-02', 
                                  '2021-09-10', *port_list)
mult_df

,AMD,CPRT
Date,,
2018-01-02,0.068093,0.009493
2018-01-03,0.051913,-0.004816
2018-01-04,0.049351,0.008066
2018-01-05,-0.019802,-0.004801
2018-01-08,0.033670,0.000459
...,...,...
2021-09-03,0.006593,0.003929
2021-09-07,-0.007005,NaN
2021-09-08,-0.027302,NaN


## Calculating Beta

Beta provides the relationship between an investment and the overall market. Risky investments tend to fall further during bad times, but will increase quicker during good times. 

Beta is found by dividing the covariance of the stock and the market by the variance of the overall market. It is a measure of systematic risk that can't be diversified away.  

$ \beta = \frac{Cov(r_x, r_m)}{\sigma_m^2} $

$ \beta = 0 $ : No relation to market

$ \beta < 1 $ : Less risky than market

$ \beta > 1 $ : More risky than the market

## Examples

Albertsons is a grocery store chain with a low beta of 0.5 because no matter what people need food and pharmecueticals. 

AMD manufacturers microchips and is a high beta stock at 1.4 because during hard times there is less demand for their products.

## Get S&P 500 and AMD Data

In [45]:
# # Will hold S&P 500 adjusted close data
# sp_df = pd.DataFrame()

# # Download data from Yahoo
# sp_df = web.DataReader('^GSPC', 'yahoo', '2017-1-3', '2021-9-10')['Adj Close']

amd_df = get_df_from_csv('AMD')

amd_df

,Adj Close,daily_return
Date,,
2017-01-03,11.430000,NaN
2017-01-04,11.430000,0.000000
2017-01-05,11.240000,-0.016623
2017-01-06,11.320000,0.007117
2017-01-09,11.490000,0.015018
...,...,...
2021-09-03,109.919998,0.006593
2021-09-07,109.150002,-0.007005
2021-09-08,106.169998,-0.027302


In [46]:
sp_df = get_df_from_csv('^GSPC')

sp_df

,Adj Close,daily_return
Date,,
2017-01-03,2257.830078,NaN
2017-01-04,2270.750000,0.005722
2017-01-05,2269.000000,-0.000771
2017-01-06,2276.979980,0.003517
2017-01-09,2268.899902,-0.003549
...,...,...
2021-09-03,4535.430176,-0.000335
2021-09-07,4520.029785,-0.003396
2021-09-08,4514.069824,-0.001319


## Add Daily Return & Save to CSV

In [124]:
# # Save S&P to csv
# save_dataframe_to_csv(sp_df, '^GSPC')

# # Get dataframe from csv
# sp_df2 = get_df_from_csv('^GSPC')

# # Add daily return to dataframe
# add_daily_return_to_df(sp_df2, '^GSPC')
# sp_df2

## Find Beta for Stock versus S&P

In [47]:
def find_beta(ticker):
    # Tickers analyzed being the S&P and the stock passed
    port_list =['^GSPC']
    port_list.append(ticker)

    mult_df = merge_df_by_column_name('daily_return',  '2018-01-02', 
                                  '2021-09-10', *port_list)
    
    # Provides the covariance between the securities
    cov = mult_df.cov() * 252
    
    # Get the covariance of the stock and the market
    cov_vs_market = cov.iloc[0,1]
    
    # Get annualized variance of the S&P
    sp_var = mult_df['^GSPC'].var() * 252
    
    # Beta is normally calculated over a 5 year period which is why you may see a difference
    beta = cov_vs_market / sp_var
    return beta


## Get Stock Beta

In [48]:
print("AMD Beta :", find_beta('AMD'))

AMD Beta : 1.4094430275300194


## Capital Asset Pricing Model

Sharpe continued to create the CAPM based on the research of Markowitz. It focuses on investments in stocks and bonds. With it we can more exactly create portfolios that match the risk an investor is willing to assume. CAPM assumes a risk free asset which of course provides a small return. So if the investor wants less risk they simply buy more of the risk free assets.

There is risk that you can limit through diversifaction (Idiosyncratic) and risk that you can't (Systematic). This portfolio contains no Idiosyncratic risk and like before it lies on the efficient frontier.

To find this portfolio we will draw a line ( The Capital Market Line ) from the Y intercept to the efficient frontier. 

Here is the formula. The securities expected return equals the risk free asset plus Beta times the market return minus the risk free asset. it is common for $ r_m - r_f $ to be considered 5% which is called the Equity Risk Premium.

$ r_i = r_f + \beta_i (r_m - r_f) $

## Calculate AMDs Expected Return

In [49]:
risk_free_rate = 0.013
ri = risk_free_rate + find_beta('AMD') * 0.05
ri

0.08347215137650096

## Sharpe Ratio

William Sharpe created the Sharpe Ratio to find the portfolio that provides the best return for the lowest amount of risk. 

*Sharpe Ratio* = $\frac{r_i - r_f}{\sigma_i}$

$r_f = $ Risk Free Rate

$r_i = $ Rate of Return of the stock

$\sigma_i = $ Standard Deviation of the Stock

As return increases so does the Sharpe Ratio, but as Standard Deviation increase the Sharpe Ratio decreases.

In [50]:
# We can find the Sharpe ratio for AMD
amd_sharpe = (ri - risk_free_rate) / (mult_df['AMD'].std() * 252 ** 0.5)
amd_sharpe

0.1253990235142509

## Get Stock Prices on Date

In [94]:
def get_prices_on_date(stocks_df, date):
    return stocks_df.loc[pd.DatetimeIndex([date])]['Adj Close'].item()

## Returns the Value of Portfolio by Date

In [60]:
def get_port_val_by_date(date, shares, tickers):
    port_prices = merge_df_by_column_name('Adj Close',  date, 
                                  date, *port_list)
    # Convert from dataframe to Python list
    port_prices = port_prices.values.tolist()
    # Trick that converts a list of lists into a single list
    port_prices = sum(port_prices, [])
    
    # Create a list of values by multiplying shares by price
    value_list = []
    for price, share in zip(port_prices, shares):
        value_list.append(price * share)
    
    return sum(value_list)

## Get Value of Portfolio at Beginning and End of Year

In [70]:
port_list = ["GNRC", "CPRT", "ODFL", "AMD", "PAYC", "CHTR", "MKC", 
             "PG", "PGR", "NEM", "CCI", "COG"]

port_shares = [25, 20, 22, 26, 1, 1, 4, 1, 5, 28, 3, 7]

# Portfolio value at start of 2020
port_val_start = get_port_val_by_date('2020-01-02', port_shares, port_list)
print("Portfolio Value at Start of 2020 : $%2.2f" % (port_val_start))

# Portfolio value at end of 2020
port_val_end = get_port_val_by_date('2020-12-31', port_shares, port_list)
print("Portfolio Value at End of 2020 : $%2.2f" % (port_val_end))

Portfolio Value at Start of 2020 : $11738.29
Portfolio Value at End of 2020 : $19232.34


## Calculate Return on Investment

ROI = $\frac{Final Value - Initial Value}{Initial Value}$

In [102]:
# Rate of return for portfolio
roi_port = (port_val_end - port_val_start) / port_val_end
print("Portfolio ROI at End of 2020 : %2.2f %%" % (roi_port * 100))

# S&P ROI
sp_df = get_df_from_csv('^GSPC')
sp_val_start = get_prices_on_date(sp_df, '2020-01-02')
sp_val_end = get_prices_on_date(sp_df, '2020-12-31')
sp_roi = (sp_val_end - sp_val_start) / sp_val_end
print("S&P ROI at End of 2020 : %2.2f %%" % (sp_roi * 100))

Portfolio ROI at End of 2020 : 38.97 %
S&P ROI at End of 2020 : 13.26 %


## Find Daily Return for Whole Portfolio

To find the daily return for the whole portfolio, I must multiply the daily price by the number of shares for each security. Then sum those values for all stocks per day. This creates a portfolio list of daily prices. Then I can calculate the daily return.

In [82]:
def get_port_daily_return(sdate, edate, shares, tickers):
    # Merge all daily prices for all stocks into 1 dataframe
    mult_df = merge_df_by_column_name('Adj Close',  sdate, 
                                  edate, *port_list)
    
    # Get the number of stocks in portfolio
    num_cols = len(mult_df.columns)
    
    # Multiply each stock column by the number of shares
    i = 0
    while i < num_cols:
        mult_df[tickers[i]] = mult_df[tickers[i]].apply(lambda x: x * shares[i])
        i += 1
        
    # Create a new column with the sums of all stocks named Total
    mult_df['Total'] = mult_df.iloc[:, 0:num_cols].sum(axis=1)
    
    # Add column for portfolio daily return
    mult_df['daily_return'] = (mult_df['Total'] / mult_df['Total'].shift(1)) - 1
    
    return mult_df

In [116]:
tot_port_df = get_port_daily_return('2020-01-02', '2020-12-31', 
                                    port_shares, port_list)
tot_port_df

,GNRC,CPRT,ODFL,AMD,PAYC,CHTR,MKC,PG,PGR,NEM,CCI,COG,Total,daily_return
Date,,,,,,,,,,,,,,
2020-01-02,2550.250053,1869.600067,2801.878052,1276.599960,271.799988,490.429993,327.721405,118.208862,341.399498,1168.771851,405.909119,115.724598,11738.293446,NaN
2020-01-03,2541.749954,1853.200073,2768.464981,1263.599960,272.709991,494.470001,332.944458,117.413841,341.353569,1158.225388,408.143738,116.060427,11668.336382,-0.005960
2020-01-06,2531.250000,1869.799957,2746.578781,1258.139984,278.089996,497.450012,334.939789,117.576675,347.554169,1169.853424,404.705795,117.000742,11672.939325,0.000394
2020-01-07,2572.500038,1867.400055,2754.457840,1254.500000,279.489990,499.160004,329.188568,116.848701,344.899635,1169.042191,397.858521,117.672400,11703.017942,0.002577
2020-01-08,2500.749969,1880.200043,2767.443634,1243.580048,283.119995,500.769989,327.428009,117.346779,346.937981,1137.943588,399.520248,113.172350,11618.212633,-0.007246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,5829.999924,2474.199982,4354.264801,2387.059937,462.130005,656.489990,369.677307,135.173843,467.704163,1664.569366,455.012192,116.410582,19372.692091,0.012335
2020-12-28,5825.999832,2534.400024,4280.906738,2381.599960,463.589996,651.900024,369.479523,136.116104,466.800270,1646.404465,461.660065,113.249495,19332.106497,-0.002095
2020-12-29,5534.249878,2518.399963,4263.775299,2356.120071,447.589996,664.979980,375.016571,135.860901,464.754562,1643.927292,460.360016,110.500713,18975.535244,-0.018445


## Find Portfolio Beta

In [109]:
def find_port_beta(port_df, sdate, edate):
    # Will hold data for S&P and my portfolio
    mult_df = pd.DataFrame()
    
    # Mask defining the dates worth of data that we want
    port_mask = (port_df.index >= sdate) & (port_df.index <= edate)
    
    # Get S&P Dataframe
    sp_df = get_df_from_csv('^GSPC')
    
    sp_mask = (sp_df.index >= sdate) & (sp_df.index <= edate)
    
    # Add S&P daily return to dataframe
    mult_df['^GSPC'] = sp_df.loc[sp_mask]['daily_return']
    
    # Add the portfolio daily return data
    mult_df['Portfolio'] = port_df.loc[port_mask]['daily_return']
    
    # Provides the covariance between the securities
    cov = mult_df.cov() * 252
    
    # Get the covariance of the stocks and the market
    cov_vs_market = cov.iloc[0,1]
    
    # Get annualized variance of the S&P
    sp_var = mult_df['^GSPC'].var() * 252
    
    # Beta is normally calculated over a 5 year period which is why you may see a difference
    beta = cov_vs_market / sp_var
    return beta

## Calculating Alpha

Alpha provides a measure of how well a portfolio has performed. The CAPM assumes an Alpha of 0. Good portfolios have a positive Alpha, while poor have negative. 

Alpha = R – Rf – beta (Rm-Rf)

* R represents the portfolio return
* Rf represents the risk-free rate of return
* Beta represents the systematic risk of a portfolio
* Rm represents the market return, per a benchmark

In [111]:
port_beta = find_port_beta(tot_port_df, '2020-01-02', '2020-12-31')
port_beta

0.9132983282237106

In [115]:
port_alpha = roi_port - risk_free_rate - (port_beta * (sp_roi - risk_free_rate))
print("Portfolio Alpha : %2.2f %%" % (port_alpha * 100))

Portfolio Alpha : 26.74 %


This means our portfolio outperformed the market in 2020 by 26.74%